## Preprocessing notebook, remove bad points

In [1]:
import sys
import os

# Get the absolute path of parent folder
current_dir = os.path.abspath("")
parent_dir = os.path.join(current_dir, os.pardir)

# Add to sys.path
sys.path.append(parent_dir)

In [2]:
import ml_combat as ml
from ml_combat import data
from ml_combat.MetaModel import MetaModel

#### Imports

In [3]:
from prophet import Prophet
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose


#### Body

In [15]:
# get csv_files in actual_submissions
csv_files = [file for file in os.listdir(ml.module_dir + '/../actual_submissions')]
csv_files += [file for file in next(os.walk(ml.module_dir))[1] if file != '__pycache__']

In [18]:
sub_dict = { # fyll på med alle
    'XGBoostComposite_laptop-30.csv': 151.15368,
    'GluonW_HPO_and_tuning_data.csv': 150.50763,
    'XGBoostComposite-150.csv': 153.74262,
    'XGBoostComposite-10.csv': 153.89125,
    'GluonW_HPO_and_tuning_data_and_sample_weighting.csv': 145.20614,
    'CompositeComposite_XGBComp_Gluon_Cat_run.csv': 147.27314,
    'XGBoostComposite_laptop-60.csv': 150.56387,
    'XGBoostComposite_metalearner_linreg.csv': 153.50807,
    'XGBoostComposite_metalearner2_linreg.csv': 152.26096
    }

In [88]:
df = ml.data.get_testing_flattened()
ret = pd.DataFrame()

for csv in sub_dict:
    temp_df = df.copy().merge(pd.read_csv(ml.module_dir + '/../actual_submissions/' + csv), left_index=True, right_index=True)
    temp_df.drop(columns=['id'], inplace=True)
    temp_df.rename(columns={'prediction': 'y_pred'}, inplace=True)
    temp_df = temp_df[['location', 'ds', 'y_pred']]
    temp_df['csv'] = csv
    temp_df['kaggle'] = sub_dict[csv]

    ret = pd.concat([ret, temp_df])

In [95]:
from plotly.subplots import make_subplots

fig = px.line()

for csv in sub_dict:
    temp = ret[ret.csv == csv].reset_index()
    fig.add_trace(px.line(temp, x='index', y='y_pred').data[0])
    # Add a scatter trace to the figure
    # fig.add_trace(
    # temp.plot()
    # fig = px.scatter(temp, x=temp.index, y='y_pred')#, labels=csv)
    # )
    # fig = px.line(temp, x=temp.index, y='y_pred', title=f'Scatter Plot for {csv}')
    # fig.update_traces(marker=dict(size=2), name=csv)
    # fig.show()
    

fig.show()

# fig.add_trace(trace1)
# fig.add_trace(trace2,secondary_y=True)



In [101]:
fig = px.scatter(ret.reset_index(), x="index", y="y_pred", color="kaggle",
                title="Automatic Labels Based on Data Frame Column Names")
fig.show()

In [76]:
ret.csv.unique()

array(['XGBoostComposite_laptop-30.csv', 'GluonW_HPO_and_tuning_data.csv',
       'XGBoostComposite-150.csv', 'XGBoostComposite-10.csv',
       'GluonW_HPO_and_tuning_data_and_sample_weighting.csv',
       'CompositeComposite_XGBComp_Gluon_Cat_run.csv',
       'XGBoostComposite_laptop-60.csv',
       'XGBoostComposite_metalearner_linreg.csv',
       'XGBoostComposite_metalearner2_linreg.csv'], dtype=object)

In [80]:
ret.groupby('csv').describe().sort_values(('kaggle', 'mean'))

y_pred              \
                                                     count        mean   
csv                                                                      
GluonW_HPO_and_tuning_data_and_sample_weighting...  2160.0  556.899730   
CompositeComposite_XGBComp_Gluon_Cat_run.csv        2160.0  555.169197   
GluonW_HPO_and_tuning_data.csv                      2160.0  534.994241   
XGBoostComposite_laptop-60.csv                      2160.0  550.024661   
XGBoostComposite_laptop-30.csv                      2160.0  549.642270   
XGBoostComposite_metalearner2_linreg.csv            2160.0  560.502332   
XGBoostComposite_metalearner_linreg.csv             2160.0  564.483668   
XGBoostComposite-150.csv                            2160.0  557.249106   
XGBoostComposite-10.csv                             2160.0  556.533917   

                                                                              \
                                                           std           min   
csv                                                                            
GluonW_HPO_and_tuning_data_and_sample_weighting...  939.905443  0.000000e+00   
CompositeComposite_XGBComp_Gluon_Cat_run.csv        951.635160  0.000000e+00   
GluonW_HPO_and_tuning_data.csv                      903.033014  0.000000e+00   
XGBoostComposite_laptop-60.csv                      959.825595  1.873327e-12   
XGBoostComposite_laptop-30.csv                      956.236319  1.651067e-12   
XGBoostComposite_metalearner2_linreg.csv            979.968571  0.000000e+00   
XGBoostComposite_metalearner_linreg.csv             982.078622  0.000000e+00   
XGBoostComposite-150.csv                            976.403661  1.866976e-12   
XGBoostComposite-10.csv                             973.760343  0.000000e+00   

                                                                           \
                                                          25%         50%   
csv                                                                         
GluonW_HPO_and_tuning_data_and_sample_weighting...  16.600872  194.578245   
CompositeComposite_XGBComp_Gluon_Cat_run.csv        14.251790  191.452330   
GluonW_HPO_and_tuning_data.csv                      12.433659  190.151640   
XGBoostComposite_laptop-60.csv                      12.762479  190.281615   
XGBoostComposite_laptop-30.csv                      13.670255  191.324250   
XGBoostComposite_metalearner2_linreg.csv            12.650929  195.244697   
XGBoostComposite_metalearner_linreg.csv             16.499853  199.594337   
XGBoostComposite-150.csv                            11.863455  192.724000   
XGBoostComposite-10.csv                             11.108189  192.415605   

                                                                             \
                                                           75%          max   
csv                                                                           
GluonW_HPO_and_tuning_data_and_sample_weighting...  629.861700  5010.349600   
CompositeComposite_XGBComp_Gluon_Cat_run.csv        616.150314  4940.610604   
GluonW_HPO_and_tuning_data.csv                      612.888262  5042.867700   
XGBoostComposite_laptop-60.csv                      593.107865  4892.138000   
XGBoostComposite_laptop-30.csv                      588.976950  4861.534700   
XGBoostComposite_metalearner2_linreg.csv            598.097965  5060.124463   
XGBoostComposite_metalearner_linreg.csv             612.380016  5148.093635   
XGBoostComposite-150.csv                            600.963150  5043.684600   
XGBoostComposite-10.csv                             596.186925  5037.510700   

                                                    kaggle             \
                                                     count       mean   
csv                                                                     
GluonW_HPO_and_tuning_data_and_sample_weighting...  2160.0  145.20614   
CompositeComposite_XGBComp_Gluon_Cat

In [99]:
ret#.head(3)

,location,ds,y_pred,csv,kaggle
0,A,2023-05-01 00:00:00,1.905078e-12,XGBoostComposite_laptop-30.csv,151.15368
1,A,2023-05-01 01:00:00,1.905078e-12,XGBoostComposite_laptop-30.csv,151.15368
2,A,2023-05-01 02:00:00,1.826851e-02,XGBoostComposite_laptop-30.csv,151.15368
3,A,2023-05-01 03:00:00,6.417390e+01,XGBoostComposite_laptop-30.csv,151.15368
4,A,2023-05-01 04:00:00,3.222504e+02,XGBoostComposite_laptop-30.csv,151.15368
...,...,...,...,...,...
2155,C,2023-07-03 19:00:00,5.593965e+01,XGBoostComposite_metalearner2_linreg.csv,152.26096
2156,C,2023-07-03 20:00:00,4.380489e+01,XGBoostComposite_metalearner2_linreg.csv,152.26096
2157,C,2023-07-03 21:00:00,1.100824e+01,XGBoostComposite_metalearner2_linreg.csv,152.26096
2158,C,2023-07-03 22:00:00,2.490878e+00,XGBoostComposite_metalearner2_linreg.csv,152.26096


In [108]:
ret.groupby(['location', 'ds']).mean().reset_index().reset_index()[['index', 'y_pred']].rename(columns={'index':'id', 'y_pred': 'prediction'}).to_csv('average_of_lt155.csv', index=False)

/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_25738/3411814484.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [109]:
ret.groupby(['location', 'ds']).mean().reset_index()

/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_25738/592086926.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,location,ds,y_pred,kaggle
0,A,2023-05-01 00:00:00,0.009665,150.900818
1,A,2023-05-01 01:00:00,0.009209,150.900818
2,A,2023-05-01 02:00:00,0.102828,150.900818
3,A,2023-05-01 03:00:00,61.578514,150.900818
4,A,2023-05-01 04:00:00,300.159679,150.900818
...,...,...,...,...
2155,C,2023-07-03 19:00:00,60.458495,150.900818
2156,C,2023-07-03 20:00:00,35.391233,150.900818
2157,C,2023-07-03 21:00:00,13.263820,150.900818
2158,C,2023-07-03 22:00:00,2.587878,150.900818


In [120]:
ret.groupby(['location','ds']).max().y_pred - ret.groupby(['location','ds']).min().y_pred

location  ds                 
A         2023-05-01 00:00:00      0.086986
          2023-05-01 01:00:00      0.082885
          2023-05-01 02:00:00      0.516599
          2023-05-01 03:00:00     12.655415
          2023-05-01 04:00:00    119.007771
                                    ...    
C         2023-07-03 19:00:00     41.896158
          2023-07-03 20:00:00     38.043775
          2023-07-03 21:00:00     24.882995
          2023-07-03 22:00:00      3.672159
          2023-07-03 23:00:00      2.741494
Name: y_pred, Length: 2160, dtype: float64

In [121]:
tp = (ret.groupby(['location','ds']).max().y_pred - ret.groupby(['location','ds']).min().y_pred).reset_index()

In [123]:
fig = px.scatter(tp.reset_index(), x="index", y="y_pred",
                title="Automatic Labels Based on Data Frame Column Names")
fig.show()

In [128]:
ret.loc[252]

,location,ds,y_pred,csv,kaggle
252,A,2023-05-28 12:00:00,1536.881600,XGBoostComposite_laptop-30.csv,151.15368
252,A,2023-05-28 12:00:00,2269.669700,GluonW_HPO_and_tuning_data.csv,150.50763
252,A,2023-05-28 12:00:00,1537.999500,XGBoostComposite-150.csv,153.74262
252,A,2023-05-28 12:00:00,1576.245100,XGBoostComposite-10.csv,153.89125
252,A,2023-05-28 12:00:00,2900.948700,GluonW_HPO_and_tuning_data_and_sample_weightin...,145.20614
252,A,2023-05-28 12:00:00,1992.780012,CompositeComposite_XGBComp_Gluon_Cat_run.csv,147.27314
252,A,2023-05-28 12:00:00,1530.670500,XGBoostComposite_laptop-60.csv,150.56387
252,A,2023-05-28 12:00:00,1730.441442,XGBoostComposite_metalearner_linreg.csv,153.50807
252,A,2023-05-28 12:00:00,1593.675684,XGBoostComposite_metalearner2_linreg.csv,152.26096
